In [16]:
!pip install pandas
!pip install openpyxl
!pip install yfiles_jupyter_graphs
!pip install networkx



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import pandas as pd
import os
from yfiles_jupyter_graphs import GraphWidget


In [13]:
notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/berber_1.xlsx")

dataframe1 = pd.read_excel(file_path)
print(dataframe1)

                                         Unnamed: 1           Epoch  \
0               NaN                             NaN           Cycle   
1               NaN                             NaN  Cycle End Rule   
2               NaN                             NaN     Visit Label   
3               NaN                             NaN       Condition   
4               NaN                             NaN    Visit Window   
5   Parent Activity                  Child Activity             BCs   
6               NaN                Informed Consent             NaN   
7               NaN  Inclusion / Exclusion Criteria             NaN   
8               NaN            Physical Examination            PE 1   
9               NaN                             NaN            PE 2   
10              NaN                             NaN          Weight   
11              NaN                             NaN          Height   

        Screening Treatment   Unnamed: 5  Unnamed: 6   Unnamed: 7  Unnamed: 

In [18]:

# shows empty widget
w = GraphWidget()
w.set_nodes([
            {'id': 0, 'properties': {'label': 'Hello World'}},
            {'id': 1, 'properties': {'label': 'This is a second node.'}}
        ])
w

GraphWidget(layout=Layout(height='500px', width='100%'))

In [17]:
from networkx import florentine_families_graph
from yfiles_jupyter_graphs import GraphWidget
w = GraphWidget()
w.import_graph(florentine_families_graph())
w


GraphWidget(layout=Layout(height='500px', width='100%'))

In [ ]:
w.circular_layout()
w